In [1]:
#import packages
import pandas as pd
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#modify import path
import sys
sys.path.append('../our_analysis')

#import our code
from hybrid_mle import fit_hybrid_mixed_model_drifting


/opt/anaconda3/envs/stan_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#Set WD
notebook_dir = os.getcwd()

In [4]:
#load dataset
df = pd.read_csv("../dataset/all_subjects.csv") #our dataset
# df = pd.read_csv("../dataset/beh_noslow.csv") #i'm fitting to replicate the paper.

#Let's remove all of the slow trials for the df
df = df[df['slow']==0]
# We need to cast the choice data to an int
df['choice1'] = df['choice1'].astype(int)
df['choice2'] = df['choice2'].astype(int)
df['final_state'] = df['final_state'].astype(int)


# ## some notes about using all_subjects:
# # If you get errors you may need to rename the fields of allsubjects becuase the original paper dataset (beh_noslow.csv) and our dataset (all_subjects) have slightly different capitalizations of headers.
# # If you get additional errors filter out the NaNs from the df before sending them to the processing function. 

# #MLE takes time the first simple run was quite fast (~10min), but expect longer depending on your CPU and the complexity of the model you're running

story_trials = df[df["condition"] == "story"]
abstract_trials=df[df["condition"] == "abstract"]

In [ ]:
story_results_df, params, logli_story_basic = fit_hybrid_mixed_model_drifting(
    data_df=story_trials,
    stan_file="../our_analysis/driftingWeight_mle.stan",
    output_file="story_params_drift.csv"
)

abstract_results_df, params, logli_abstract_basic = fit_hybrid_mixed_model_drifting(
    data_df=abstract_trials,
    stan_file="../our_analysis/driftingWeight_mle.stan",
    output_file="abstract_params_drift.csv"
)

In [6]:
#store the results in a clean table we can use for the assignment
results =[] 

for df in [story_results_df, abstract_results_df]:
    params=df.iloc[0].to_dict()
    results.append(params)

results[0]['logli']=logli_story_basic
results[1]['logli']=logli_abstract_basic

results_df = pd.DataFrame(results)
results_df.head()
## we should probably just save this as a csv here. It has the participant field which we can drop in later formatting.
results_df.to_csv("hybrid_model_params_driftingW.csv", index=False)
